In [1]:
import json
import pandas as pd
import numpy as np
import sys
import csv
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns


csv.field_size_limit(sys.maxsize)

131072

In [4]:
chunks = pd.read_csv("/content/drive/MyDrive/yelp_data_ready1.csv", engine='python', chunksize = 500000)
counter = 1
for c in chunks:
  if counter == 1: 
    print(counter)
    df = c.copy()
    counter += 1
  else: 
    print(counter)
    df = pd.concat([df,c])
    counter += 1
  #Chunk is like a list of dataframes

  

1
2
3
4
5
6
7
8
9
10


In [6]:
df = df.drop(['useful', 'funny', 'cool'], axis=1)
df = df.drop(['is_vegan_restaurant', 'is_vegetarian_restaurant', 'is_omnivore_restaurant', 'biz_veg_code', 'user_vegan', 'user_vegetarian', 'user_omnivore', 'user_veg_code'], axis=1)

In [14]:
df.head()

,Unnamed: 0,review_id,user_id,business_id,stars_x,date,review_year,review_month,review_date,city,...,categories,is_restaurant,is_vegan_restaurant,is_vegetarian_restaurant,is_omnivore_restaurant,biz_veg_code,user_vegan,user_vegetarian,user_omnivore,user_veg_code
0,0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11,2018,7,7,North Wales,...,"Restaurants, Breakfast & Brunch, Food, Juice B...",1,0,0,1,Omnivore,0.000000,0.000000,1.000000,Omnivore
1,1,VJxlBnJmCDIy8DFG0kjSow,Iaee7y6zdSB3B-kRCo4z1w,XQfwVwDr-v0ZS3_CbbE5Xw,2,2017-05-13 17:06:55,2017,5,13,North Wales,...,"Restaurants, Breakfast & Brunch, Food, Juice B...",1,0,0,1,Omnivore,0.000000,0.000000,1.000000,Omnivore
2,2,S6pQZQocMB1WHMjTRbt77A,ejFxLGqQcWNLdNByJlIhnQ,XQfwVwDr-v0ZS3_CbbE5Xw,4,2017-08-08 00:58:18,2017,8,8,North Wales,...,"Restaurants, Breakfast & Brunch, Food, Juice B...",1,0,0,1,Omnivore,0.020619,0.041237,0.938144,Omnivore
3,3,WqgTKVqWVHDHjnjEsBvUgg,f7xa0p_1V9lx53iIGN5Sug,XQfwVwDr-v0ZS3_CbbE5Xw,3,2017-11-19 02:20:23,2017,11,19,North Wales,...,"Restaurants, Breakfast & Brunch, Food, Juice B...",1,0,0,1,Omnivore,0.018692,0.018692,0.962617,Omnivore
4,4,M0wzFFb7pefOPcxeRVbLag,dCooFVCk8M1nVaQqcfTL3Q,XQfwVwDr-v0ZS3_CbbE5Xw,2,2017-09-09 17:49:47,2017,9,9,North Wales,...,"Restaurants, Breakfast & Brunch, Food, Juice B...",1,0,0,1,Omnivore,0.000000,0.000000,1.000000,Omnivore


In [8]:
#Coding biz as vegan or vegetarian restaurant or not
#Coding as vegan first to preempt vegetarian restaurants overlapping them
is_vegan_restaurant = []
for item in df['categories']:
  if item == None:
    is_vegan_restaurant.append(0)
  elif item.find('Vegan') != -1:
    is_vegan_restaurant.append(1)
  else: 
    is_vegan_restaurant.append(0)

df['is_vegan_restaurant'] = is_vegan_restaurant

is_vegetarian_restaurant = []
for item in df['categories']:
  if item == None:
    is_vegetarian_restaurant.append(0)
  elif item.find('Vegan') != -1:
    is_vegetarian_restaurant.append(0)
  elif item.find('Vegetarian') != -1:
    is_vegetarian_restaurant.append(1)
  else: 
    is_vegetarian_restaurant.append(0)

df['is_vegetarian_restaurant'] = is_vegetarian_restaurant

def is_omnivore(is_vegetarian,is_vegan):
   if is_vegetarian or is_vegan:
     return 0
   return 1

df['is_omnivore_restaurant'] = df.apply(lambda x: is_omnivore(x['is_vegetarian_restaurant'],x['is_vegan_restaurant']), axis = 1)

def veg_code(is_vegetarian,is_vegan):
  if is_vegetarian:
    return 'Vegetarian'
  if is_vegan:
    return 'Vegan'
  return 'Omnivore'

df['biz_veg_code'] = df.apply(lambda x: veg_code(x['is_vegetarian_restaurant'],x['is_vegan_restaurant']), axis = 1)

In [9]:
#Assuming that veg*ns have >15% of the have the word vegan or vegetarian work?  
#How do I get all the reviews within/by a given user?

def veg_quant(x_vegetarian,x_vegan):
  if ((x_vegetarian >= 0.15) & (x_vegan >= 0.15)):
    return 'Vegan'
  elif ((x_vegetarian < 0.15) & (x_vegan >= 0.15)):
    return 'Vegan'
  elif ((x_vegetarian >= 0.15) & (x_vegan < 0.15)):
    return 'Vegetarian'
  else:
    return 'Omnivore'


#user_id is the reviewer column
#Text is the review content column

#Need to do the same for trends_df

#proportion of reviews by veg_code
user_veg_code = df.groupby(['user_id'])[['is_vegan_restaurant', 'is_vegetarian_restaurant','is_omnivore_restaurant']].mean()


#biz_veg_code = trends_df.groupby(['business_id'])[['vegan_review', 'vegetarian_review','omnivore_review']].mean()

user_veg_code.columns = ['user_vegan', 'user_vegetarian', 'user_omnivore']
#biz_veg_code.columns = ['biz_vegan', 'biz_vegetarian', 'biz_omnivore']

In [11]:
df = pd.merge(df, user_veg_code, how = 'left', on = 'user_id')


In [13]:
df['user_veg_code'] = df.apply(lambda x: veg_quant(x['user_vegetarian'],x['user_vegan']), axis = 1)


In [15]:
#Removing entries before 2012 and after 2021 due to small sample sizes.
df = df[df['review_year'] > 2011]
df = df[df['review_year'] < 2022]

In [16]:
df.biz_veg_code.value_counts()

Omnivore      4121881
Vegan          110708
Vegetarian     106617
Name: biz_veg_code, dtype: int64

In [17]:
df.user_veg_code.value_counts()

Omnivore      4027098
Vegan          171693
Vegetarian     140415
Name: user_veg_code, dtype: int64

In [18]:
import os

fname = 'yelp_data_ready3.csv'

directory = './Desktop'
if not os.path.exists(directory):
    os.mkdir(directory)

path = os.path.join(directory, fname)    

df.to_csv(path)

#Path is : /content/Desktop/yelp_data_ready3.csv

In [ ]:
#Method for reading in random rows for sentiment analysis
#import random

#filename = "/content/drive/MyDrive/yelp_review.csv"
#n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
#s = 500000 #desired sample size
#skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
#df = pd.read_csv(filename, skiprows=skip) 

In [ ]:
#from google.colab import files
#uploaded = files.upload()

#import io
#df2 = pd.read_csv(io.BytesIO(uploaded['yelp_business_attributes.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
#SOURCE FILES

#File #1
biz_attr_df = pd.read_json("/content/drive/MyDrive/yelp_academic_dataset_business.json", lines=True)

pd.set_option('display.max_columns', None)

biz_attr_df = biz_attr_df.drop(['name','address', 'attributes','hours'],axis=1)
biz_attr_df.head()

In [ ]:
#Coding biz as restaurant or not
is_restaurant = []
for item in biz_attr_df['categories']:
  if item == None:
    is_restaurant.append(0)
  elif item.find('Restaurants') != -1:
    is_restaurant.append(1)
  else: 
    is_restaurant.append(0)

biz_attr_df['is_restaurant'] = is_restaurant
#Removing non-restaurants
biz_attr_df = biz_attr_df.loc[biz_attr_df['is_restaurant'] == 1,:]
biz_attr_df.shape

In [ ]:
#Coding biz as vegan or vegetarian restaurant or not
#Coding as vegan first to preempt vegetarian restaurants overlapping them
is_vegan_restaurant = []
for item in biz_attr_df['categories']:
  if item == None:
    is_vegan_restaurant.append(0)
  elif item.find('Vegan') != -1:
    is_vegan_restaurant.append(1)
  else: 
    is_vegan_restaurant.append(0)

biz_attr_df['is_vegan_restaurant'] = is_vegan_restaurant

is_vegetarian_restaurant = []
for item in biz_attr_df['categories']:
  if item == None:
    is_vegetarian_restaurant.append(0)
  elif item.find('Vegan') != -1:
    is_vegetarian_restaurant.append(0)
  elif item.find('Vegetarian') != -1:
    is_vegetarian_restaurant.append(1)
  else: 
    is_vegetarian_restaurant.append(0)

biz_attr_df['is_vegetarian_restaurant'] = is_vegetarian_restaurant

def is_omnivore(is_vegetarian,is_vegan):
   if is_vegetarian or is_vegan:
     return 0
   return 1

biz_attr_df['is_omnivore_restaurant'] = biz_attr_df.apply(lambda x: is_omnivore(x['is_vegetarian_restaurant'],x['is_vegan_restaurant']), axis = 1)

def veg_code(is_vegetarian,is_vegan):
  if is_vegetarian:
    return 'Vegetarian'
  if is_vegan:
    return 'Vegan'
  return 'Omnivore'

biz_attr_df['biz_veg_code'] = biz_attr_df.apply(lambda x: veg_code(x['is_vegetarian_restaurant'],x['is_vegan_restaurant']), axis = 1)

In [ ]:
#Counting number of restaurants coded as both vegetarian and vegan

counter = 0
for item in biz_attr_df['categories']:
  if item == None:
    counter += 0
  elif((item.find('Vegan') != -1) and (item.find('Vegetarian') != -1)):
    counter += 1
  else: 
        counter += 0

print(counter)

In [ ]:
#Checking to see if coding worked
#df[(df['is_vegetarian_restaurant'] == 1) & (df['is_vegan_restaurant'] ==1)]

In [ ]:
counter = 0
for item in biz_attr_df['categories']:
  if item == None:
    counter += 0
  elif(item.find('Vegetarian') != -1):
    counter += 1
  else: 
        counter += 0

print(counter)

In [ ]:
#Splitting up the 5261668 rows in yelp_review.csv into three chunks for easier processing
chunks = pd.read_json("/content/drive/MyDrive/yelp_academic_dataset_review.json", chunksize = 500000, lines = True)
counter = 1
for c in chunks:
  if counter == 1: 
    print(counter)
    df = c.copy()
    counter += 1
  else: 
    print(counter)
    df = pd.concat([df,c])
    counter += 1

#reviews_df1 = pd.read_csv("/content/drive/MyDrive/yelp_review.csv", sep=',', engine='python', nrows = 2000000)
# Dataset is now stored in a Pandas Dataframe

In [ ]:
#Create columns with year, month, and day for graphing
df['review_year'] = df['date'].dt.year
df['review_month'] = df['date'].dt.month
df['review_date'] = df['date'].dt.day

In [ ]:
#Optional to conserve memory: drop text column
df = df.drop(['text'], axis=1)

In [ ]:
df = df.drop(['useful', 'funny', 'cool'], axis=1)

In [ ]:
#searching through reviews for "vegan" as proxy for vegan restaurants - total = 41819 instances of 'vegan'
np.sum(df['text'].str.contains("vegan", case=False, flags=0, na=None, regex = False))

In [ ]:
#searching through reviews for "vegetarian" as proxy for vegetarian restaurants - total = 57976 instances of 'vegetarian'
np.sum(df['text'].str.contains("vegetarian", case=False, flags=0, na=None, regex = False))


In [ ]:
biz_reviews_attrs = pd.merge(df, biz_attr_df, how = 'inner', on = 'business_id' )
biz_reviews_attrs.head(20)

In [ ]:
#Ignore for main analyses
#Transforming the date column to datetime, not adding:

#biz_reviews_attrs['date'] = pd.to_datetime(biz_reviews_attrs['date'])

In [ ]:
#Datetimes are all ints now in their new columns
#type(biz_reviews_attrs['review_date'][1])

In [ ]:
import os

fname = 'yelp_data_ready2.csv'

directory = './Desktop'
if not os.path.exists(directory):
    os.mkdir(directory)

path = os.path.join(directory, fname)    

biz_reviews_attrs.to_csv(path)

#Path is : /content/Desktop/yelp_data_ready.csv

In [ ]:
!ls Desktop

In [ ]:
# restaurants['omnivore_review'] = restaurants.apply(lambda x: is_omnivore(x['vegetarian_review'],x['vegan_review']), axis = 1)

In [ ]:
# def veg_code(is_vegetarian,is_vegan):
#   if is_vegetarian:
#     return 'Vegetarian'
#   if is_vegan:
#     return 'Vegan'
#   return 'Omnivore'

In [ ]:
# restaurants['review_veg_code'] = restaurants.apply(lambda x: veg_code(x['vegetarian_review'],x['vegan_review']), axis = 1)

In [ ]:
#Full text entries are only viewable one at a time
biz_reviews_attrs['text'][3]

In [ ]:
#Make a graph of how ratings of vegetarian/vegan trends_df differ if reviewer is veg versus omni (imagine 3 separate panels )
#What is a proxy for user diet status?  Would assuming that if >15% of their comments have the word vegan or vegetarian work?  
#How do I get all the reviews within/by a given user?

def veg_quant(x_vegetarian,x_vegan):
  if ((x_vegetarian >= 0.15) & (x_vegan >= 0.15)):
    return 'Vegan'
  elif ((x_vegetarian < 0.15) & (x_vegan >= 0.15)):
    return 'Vegan'
  elif ((x_vegetarian >= 0.15) & (x_vegan < 0.15)):
    return 'Vegetarian'
  else:
    return 'Omnivore'


#user_id is the reviewer column
#Text is the review content column

#Need to do the same for trends_df

#proportion of reviews by veg_code
user_veg_code = biz_reviews_attrs.groupby(['user_id'])[['is_vegan_restaurant', 'is_vegetarian_restaurant','is_omnivore_restaurant']].mean()


#biz_veg_code = trends_df.groupby(['business_id'])[['vegan_review', 'vegetarian_review','omnivore_review']].mean()

user_veg_code.columns = ['user_vegan', 'user_vegetarian', 'user_omnivore']
#biz_veg_code.columns = ['biz_vegan', 'biz_vegetarian', 'biz_omnivore']


biz_reviews_attrs = pd.merge(biz_reviews_attrs, user_veg_code, how = 'left', on = 'user_id')
#trends_df = pd.merge(trends_df, biz_veg_code, how = 'left', on = 'business_id')

biz_reviews_attrs.head()








# def reviewer_code(user_id,text):
#   if ('vegetarian' in text & 'vegan' in text):
#     return 'vegan'

#   elif ('vegetarian' in text):
#     return 'vegetarian'

#   elif ('vegan' in text):
#     return 'vegan'

#   else: return 'omnivore'


In [ ]:
biz_reviews_attrs['user_veg_code'] = biz_reviews_attrs.apply(lambda x: veg_quant(x['user_vegetarian'],x['user_vegan']), axis = 1)


#fig3 = px.line(data_frame = grouped_df, x='review_year', y='stars', line_group = ['user_vegan', 'user_vegetarian','user_omnivore'], color = 'user_veg_code')



In [ ]:
#Lightweight NLP 

#Average NLP sentiment score over time/by year
from wordcloud import WordCloud
#Wordcloud on positive and negative reviews

!pip install transformers

# Import libraries
from transformers import pipeline

# Create an NLP pipeline that runs sentiment analysis
classifier = pipeline("sentiment-analysis")

# Classify some text (this may download some data and models the first time you run it)
#sentiment_1 = classifier("I hate machine learning! It's the absolute worst.")
#sentiment_2 = classifier("Machine learning is the absolute bees knees I love it so much!")

# Print sentiment output
#print(sentiment_1, sentiment_2)

In [ ]:
restaurants['sentiment'] = [classifier("".join(text.split()[:512])) for text in restaurants['text']]

In [ ]:
max(restaurants['text'].agg(len))

In [ ]:
#Print text that fails running through sentiment analysis:

for text in restaurants['text']:
    try:
        classifier("".join(text.split()[:512]))
    except:
        print(text) 
        raise ValueError

In [ ]:
large_txt = """ 
I think I may have been let down by this place because of all the rave reviews. But the fact is, I've eaten here twice and neither time was spectacular. 

My expectations of a breakfast place are cheap prices, good food (because eggs are hard to mess up), fast turnover and service. This place has prices on the high end of breakfast place, on weekends they are very crowded and it will take some time to be seated and the food is alright. 

The last time I went, I was with my husband and we went in the door to a huge crowd. There was no wait to be seated sign, but we guessed correctly and waited. Problem is that you pay and wait in the same place, which meant people were coming and going in a very small area. It was chaos for awhile. Finally we were seated outside at a picnic table and told that other people may be seated with us at the table, because they lacked space. 

The waitress took some time to arrive and by then we were ready to give our order. I ordered eggs and bacon and toast. We also each ordered fruit smoothies. I asked for mine without banana. The waitress came back a few minutes later and said with a straight face, "The guys back there put like three bananas in your smoothie!" 

It took me a minute to realize she was joking....heh....

We played several rounds of Scrabble on an iphone because the food was taking quite awhile and it finally came out. Our drinks were no where to be seen. We had to ask twice for them. By then we were almost done the food!

The food was alright, but I usually go to places like Egg Smart and for the cost, they give you way more food, for a better price and honestly, I think it tastes way better. These famous "Boom frites" are just thick, cut potatoes and they give you one slice of toast! 

Anyway, if you're in the area and really want some breakfast, it's not terrible, but I think you can do way better! 

Oh, and I disagree that it's a good place for a group. Unless you want to wait forever...or make a group by joining another couple at their picnic table!

"""

In [ ]:
#Note that the classifier evaluates the text above as positive!  This is concerning for quality issues.

classifier(large_txt)

In [ ]:
results = []
for text in restaurants['text']:
    try:
        results.append(classifier("".join(text.split()[:512])))
    except Exception as e:
        results.append(None)
        print(e) 

In [ ]:
classifier(large_txt[:512])

In [ ]:
restaurants['sentiment'] = results

In [ ]:
#Score of classifier is confidence 
#Can compare counts of 'Positive' and 'Negative' - but this may be unreliable.
#Note that the example of "large_txt" gives the wrong sentiment at high confidence

In [ ]:
import re

# Convert all the string to lower cases
restaurants['text'] = restaurants['text'].str.lower()

# \s+ means all empty space (\n, \r, \t)
restaurants['text'] = restaurants['text'].apply(lambda x: re.sub('\s+', ' ', x))

#remove punctuation
restaurants['text'] = restaurants['text'].apply(lambda x: re.sub('[^\w\s]', '', x))

import nltk

# Uncomment the following line the first time you run the code
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


from nltk.corpus import stopwords
stop = stopwords.words('english')
#print(stop)

#removing stopwords
restaurants['text'] = restaurants['text'].apply(lambda text: " ".join(word for word in text.split() if word not in stop))

In [ ]:
wc = WordCloud(background_color="white", max_words=2000, width=800, height=400)
# generate vegan word cloud
wc.generate(' '.join(restaurants[restaurants['biz_veg_code'] == 'Vegan']['text']))

# show
plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wc2 = WordCloud(background_color="white", max_words=2000, width=800, height=400)
# generate vegetarian word cloud
wc2.generate(' '.join(restaurants[restaurants['biz_veg_code'] == 'Vegetarian']['text']))

# show
plt.figure(figsize=(12, 6))
plt.imshow(wc2, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#This file is too big to run on the merged data.  

wc3 = WordCloud(background_color="white", max_words=2000, width=800, height=400)
# generate vegan word cloud
wc3.generate(' '.join(restaurants[restaurants['biz_veg_code'] == 'Omnivore']['text']))
# show
plt.figure(figsize=(12, 6))
plt.imshow(wc3, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Failed attempt to run wordcloud on 1/3 of the data:

#DO NOT RUN
reviews_df1 = reviews_df1.loc[reviews_df1['text'].str.contains("food", case=False, flags=0, na=None, regex = False)]
reviews_df1['vegetarian_review'] = reviews_df1['text'].str.contains("vegetarian", case=False, flags=0, na=None, regex = False)
reviews_df1['vegan_review'] = reviews_df1['text'].str.contains("vegan", case=False, flags=0, na=None, regex = False)

reviews_df1['review_veg_code'] = reviews_df1.apply(lambda x: veg_code(x['vegetarian_review'],x['vegan_review']), axis = 1)
reviews_df1['omnivore_review'] = reviews_df1.apply(lambda x: is_omnivore(x['vegetarian_review'],x['vegan_review']), axis = 1)

user_veg_code = reviews_df1.groupby(['user_id'])[['vegan_review', 'vegetarian_review','omnivore_review']].mean()
biz_veg_code = reviews_df1.groupby(['business_id'])[['vegan_review', 'vegetarian_review','omnivore_review']].mean()

user_veg_code.columns = ['user_vegan', 'user_vegetarian', 'user_omnivore']
biz_veg_code.columns = ['biz_vegan', 'biz_vegetarian', 'biz_omnivore']

reviews_df1 = pd.merge(reviews_df1, user_veg_code, how = 'left', on = 'user_id')
reviews_df1 = pd.merge(reviews_df1, biz_veg_code, how = 'left', on = 'business_id')



reviews_df1['user_veg_code'] = reviews_df1.apply(lambda x: veg_quant(x['user_vegetarian'],x['user_vegan']), axis = 1)
reviews_df1['biz_veg_code'] = reviews_df1.apply(lambda x: veg_quant(x['biz_vegetarian'],x['biz_vegan']), axis = 1)

reviews_df1.head()

from wordcloud import WordCloud

wc4 = WordCloud(background_color="white", max_words=2000, width=800, height=400)
wc4.generate(' '.join(reviews_df1[reviews_df1['biz_veg_code'] == 'Omnivore']['text']))

In [ ]:
#TRYNG SENTIMENT ANALYSIS WITH RANDOMLY SAMPLED SUBSET OF DATA

df['date'] = pd.to_datetime(df['date'])

df['review_year'] = df['date'].dt.year
df['review_month'] = df['date'].dt.month
df['review_day'] = df['date'].dt.day

#merged_reviews_123.loc[merged_reviews_123['text'].str.contains("food", case=False, flags=0, na=None, regex = False)]
df['vegetarian_review'] = df['text'].str.contains("vegetarian", case=False, flags=0, na=None, regex = False)
df['vegan_review'] = df['text'].str.contains("vegan", case=False, flags=0, na=None, regex = False)

df['review_veg_code'] = df.apply(lambda x: veg_code(x['vegetarian_review'],x['vegan_review']), axis = 1)
df['omnivore_review'] = df.apply(lambda x: is_omnivore(x['vegetarian_review'],x['vegan_review']), axis = 1)

user_veg_code = df.groupby(['user_id'])[['vegan_review', 'vegetarian_review','omnivore_review']].mean()
biz_veg_code = df.groupby(['business_id'])[['vegan_review', 'vegetarian_review','omnivore_review']].mean()

user_veg_code.columns = ['user_vegan', 'user_vegetarian', 'user_omnivore']
biz_veg_code.columns = ['biz_vegan', 'biz_vegetarian', 'biz_omnivore']

df = pd.merge(df, user_veg_code, how = 'left', on = 'user_id')
df = pd.merge(df, biz_veg_code, how = 'left', on = 'business_id')

df['user_veg_code'] = df.apply(lambda x: veg_quant(x['user_vegetarian'],x['user_vegan']), axis = 1)
df['biz_veg_code'] = df.apply(lambda x: veg_quant(x['biz_vegetarian'],x['biz_vegan']), axis = 1)

In [ ]:
import re

# Convert all the string to lower cases
df['text'] = df['text'].str.lower()

# \s+ means all empty space (\n, \r, \t)
df['text'] = df['text'].apply(lambda x: re.sub('\s+', ' ', x))

#remove punctuation
df['text'] = df['text'].apply(lambda x: re.sub('[^\w\s]', '', x))

import nltk

# Uncomment the following line the first time you run the code
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')


from nltk.corpus import stopwords
stop = stopwords.words('english')
#print(stop)

#removing stopwords
df['text'] = df['text'].apply(lambda text: " ".join(word for word in text.split() if word not in stop))

In [ ]:
from wordcloud import WordCloud

wc4 = WordCloud(background_color="white", max_words=2000, width=800, height=400)
wc4.generate(' '.join(df[df['biz_veg_code'] == 'Omnivore']['text']))

# show
plt.figure(figsize=(12, 6))
plt.imshow(wc4, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
results = []
for text in df['text']:
  results.append(classifier(text))

In [ ]:
results[0:5]

In [ ]:
len(df['text'][3])

In [ ]:
classifier(df['text'][3])

In [ ]:
len(df['text'][2].split(' '))

In [ ]:
#String split the text column
#If length > 512, index it from 0 to 511, else return original list
#convert list to string
#run classifier on reduced text column (it only works on strings, not lists)